# Random Forests 1

---

*Goals*

- Build random forest models
- Grid search with cross validation
- Condiser the 12 representations
- Save results for further analysis

*Features*



## Setup

In [1]:
import os
import re
import time
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_matrix
from datetime import datetime
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

dt_object = datetime.fromtimestamp(time.time())
day, T = str(dt_object).split('.')[0].split(' ')
print('Revised on: ' + day)

Revised on: 2021-01-01


## Load Data

In [2]:
# load target
raw_path = os.path.join("data","1_raw")
filename = "y_train.csv"
y = pd.read_csv(os.path.join(raw_path, filename))
y = np.array(y.iloc[:,0].ravel())
y[y=='ham'] = 0
y[y=='spam'] = 1
y = y.astype('int')

# load 12 matrices
proc_dir = os.path.join("data","2_processed")
Xnames = [x for x in os.listdir(proc_dir) if re.search('.npz', x)]
Xs = []
for i, X in enumerate(Xnames):
    path_ = os.path.join(proc_dir, Xnames[i])
    Xs.append(sp.load_npz(path_))

## Grid search radnom forest models optimizing sensitivity

I struggled with [Scikit-Learn's GridSearchCV](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/ensemble/_forest.py#L883) and my own DIY grid searches trying to optimize for sensitivity until I stubled upon Kevin Arvai's fantastic tutorial [Fine tuning a classifier in scikit-learn](https://towardsdatascience.com/fine-tuning-a-classifier-in-scikit-learn-66e048c21e65). 

Kevin Arvai introduces two strategies:

1. GridSearchCV using the scoring argument
2. Adjust the decision threshold to identify the operating point


## Strategy 1: GridSearchCV


### Using the Bag-of-Trigrams

In [3]:
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer
from sklearn.metrics import recall_score, accuracy_score, precision_score, confusion_matrix

# 1. X_bot
X_bot = Xs[0].toarray()

X_train, X_test, y_train, y_test = train_test_split(X_bot, y, stratify=y)

def check_target_distro(y_train, y_test):
    train_pos = y_train.sum() / len(y_train)
    test_pos = y_test.sum() / len(y_test)
    train_neg = 1 - train_pos
    test_neg = 1 - train_pos
    return {'train_pos':train_pos.round(3),
            'test_pos':test_pos.round(3),
            'train_neg':train_neg.round(3),
            'test_neg':test_neg.round(3)}

check_target_distro(y_train, y_test)

{'train_pos': 0.133, 'test_pos': 0.132, 'train_neg': 0.867, 'test_neg': 0.867}

In [4]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1, random_state=42)

param_grid = {
    'min_samples_split': [5, 10, 20],
    'n_estimators' : [50, 100, 200],
    'max_depth': [3, 5, 10], 
    'max_features': [10, 25, 50, 100, 200] # mtry
}

scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score)
}

In [5]:
def gridsearch_wrapper(cv=5, refit_score='accuracy_score', n_jobs=-1):
    """Fits a GridSearchCV classifier using refit_score for optimization
       Prints classifier's performance metrics
    """
    T1 = time.time()
    cv_folds = StratifiedKFold(n_splits=cv)
    grid_search = GridSearchCV(clf, param_grid, scoring=scorers, refit=refit_score, 
                               cv=cv_folds, return_train_score=True, n_jobs=n_jobs)
    
    grid_search.fit(X_train, y_train)
    
    # make the predictions
    y_pred = grid_search.predict(X_test)
    print('Best params for {}'.format(refit_score))
    print(grid_search.best_params_)
    
    # confusion matrix on the test data
    print('\nConfusion matrix of Random Forest optimized for {} on the test data:'.format(refit_score))
    print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                       columns=['pred_neg', 'pred_pos'],
                       index=['neg', 'pos']))
    
    mins, secs = divmod(time.time() - T1, 60)
    print(f'\nElapsed: {mins:0.0f} m {secs:0.0f} s')
    return grid_search

In [11]:
def format_results(gridsearch, sort_by):
    """Format results, returning top 6 given a sorting score.
    """
    res_df = pd.DataFrame(gridsearch.cv_results_)
    res_df = res_df[['mean_test_precision_score', 'mean_test_recall_score', 'mean_test_accuracy_score', 
                     'param_max_depth', 'param_max_features', 'param_min_samples_split', 'param_n_estimators']]
    res_df = res_df.sort_values(by=sort_by, ascending=False).round(3).head()
    return res_df

### TEST: Optimizing for Precision

First I copy the example in Kevin Arvai's tutorial, proving that the gridsearch wrapper does indeed optimize for the correct score, by opitmizing for the wrong one and seeing it fail.

In [7]:
gridsearch_precision = gridsearch_wrapper(refit_score='precision_score')

Best params for precision_score
{'max_depth': 3, 'max_features': 25, 'min_samples_split': 5, 'n_estimators': 50}

Confusion matrix of Random Forest optimized for precision_score on the test data:
     pred_neg  pred_pos
neg       846         0
pos       124         5

Elapsed: 5 m 45 s


In [12]:
format_results(gridsearch_precision, 'mean_test_recall_score')

,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
128,0.991,0.833,0.977,10,200,5,200
127,0.988,0.827,0.976,10,200,5,100
133,0.991,0.827,0.976,10,200,20,100
132,0.988,0.827,0.976,10,200,20,50
126,0.988,0.825,0.975,10,200,5,50


In [13]:
y_pred = gridsearch_precision.predict(X_test)

print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                   columns=['pred_neg', 'pred_pos'],
                   index=['neg', 'pos']))

     pred_neg  pred_pos
neg       846         0
pos       124         5


In [15]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
recall = tp / (tp + fn)
print(f'sensitivity: {recall:0.4f}')

sensitivity: 0.0388


### Optimize for Sensitivity

Here I do the correct thing and optimize for sensitivity, which is what we need.

In [16]:
gridsearch_recall = gridsearch_wrapper(refit_score='recall_score')

Best params for recall_score
{'max_depth': 10, 'max_features': 200, 'min_samples_split': 5, 'n_estimators': 200}

Confusion matrix of Random Forest optimized for recall_score on the test data:
     pred_neg  pred_pos
neg       842         4
pos        15       114

Elapsed: 5 m 49 s


In [17]:
format_results(gridsearch_recall, 'mean_test_recall_score')

,mean_test_precision_score,mean_test_recall_score,mean_test_accuracy_score,param_max_depth,param_max_features,param_min_samples_split,param_n_estimators
128,0.991,0.833,0.977,10,200,5,200
127,0.988,0.827,0.976,10,200,5,100
133,0.991,0.827,0.976,10,200,20,100
132,0.988,0.827,0.976,10,200,20,50
126,0.988,0.825,0.975,10,200,5,50


In [23]:
y_pred = gridsearch_recall.predict(X_test)

print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                   columns=['pred_neg', 'pred_pos'],
                   index=['neg', 'pos']))

     pred_neg  pred_pos
neg       842         4
pos        15       114


In [24]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
recall = tp / (tp + fn)
print(f'sensitivity: {recall:0.4f}')

sensitivity: 0.8837


In [27]:
param_grid_deep = {
    'min_samples_split': [5, 10, 20],
    'n_estimators' : [300, 500],
    'max_depth': [3, 5, 10, 25], 
    'max_features': [100, 250, 500] # mtry
}

In [ ]:
gridsearch_recall_deep = gridsearch_wrapper(refit_score='recall_score')

In [ ]:
format_results(gridsearch_recall_deep, 'mean_test_recall_score')

In [ ]:
y_pred = gridsearch_recall_deep.predict(X_test)

print(pd.DataFrame(confusion_matrix(y_test, y_pred),
                   columns=['pred_neg', 'pred_pos'],
                   index=['neg', 'pos']))

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
recall = tp / (tp + fn)
print(f'sensitivity: {recall:0.4f}')

---

## DIY GridSearchCV - deprecated...

I created this DIY grid search before trying to implement the same using sklearn's GridSearchCV which is probably faster but more complex.

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, recall_score

def scikitlearn_cv(clf, X, y, seed_, cv=5, test_size=.25, n_jobs=-1):
    # setup scorer for each metric
    scorer_ = {
        'acc': make_scorer(accuracy_score),
        'tpr': make_scorer(recall_score, pos_label=1),
        'tnr': make_scorer(recall_score, pos_label=0)
    }
    
    # return mean cv score for each metric
    acc = cross_val_score(clf, X, y, cv=cv, scoring=scorer_['acc'], n_jobs=n_jobs)
    tpr = cross_val_score(clf, X, y, cv=cv, scoring=scorer_['tpr'], n_jobs=n_jobs)
    tnr = cross_val_score(clf, X, y, cv=cv, scoring=scorer_['tnr'], n_jobs=n_jobs)
    return acc.mean(), tpr.mean(), tnr.mean()

def collect_cvs(clf, Xs, Xnames, y, seed_, cv=5, test_size=.25):
    # instantiate lists
    accs, tprs, tnrs, secs = [], [], [], []
    
    # cross-validate and collect metrics
    for X in Xs:
        start_cv = time.time()
        acc, tpr, tnr = scikitlearn_cv(clf, X, y, 
                                       seed_=seed_, cv=cv, test_size=test_size)
        accs.append(round(acc, 4))
        tprs.append(round(tpr, 4))
        tnrs.append(round(tnr, 4))
        secs.append(round(time.time() - start_cv, 1))
        
    # create dictionary
    data = {'representation': Xnames,
            'mean_accuracy': accs,
            'mean_sensitivity': tprs, 
            'mean_specificity': tnrs,
            'elapsed_seconds':secs}
    return data

def grid_search(Xs, Xnames, y, cv_seed, rf_seed, 
                n_estimators, max_features, max_samples, max_depth, max_leaf_nodes, 
                cv=5, n_jobs=-1):
    start_ = time.time()
    
    # instantiate list of data frames
    list_of_dfs = []
    # collect cv metrics for each mtry value
    for mtry in max_features:
        rf_clf = RandomForestClassifier(random_state=rf_seed,
                                        n_estimators=n_estimators,
                                        max_features=mtry,
                                        max_samples=max_samples,
                                        max_depth=max_depth,
                                        max_leaf_nodes=max_leaf_nodes,
                                        n_jobs=n_jobs,
                                        verbose=0)        
        data = collect_cvs(rf_clf, Xs, Xnames, y, seed_=cv_seed, cv=cv)
        df = pd.DataFrame(data)
        df['mtry'] = mtry        
        list_of_dfs.append(df)
        
    # flatten list of data frames and reset index
    flattened_df = pd.concat(list_of_dfs)
    ix_num = len(max_features) * len(Xs)
    flattened_df.index = range(ix_num)
    
    mins, secs = divmod(time.time() - start_, 60)
    print(f'Elapsed: {mins:0.0f} m {secs:0.0f} s')
    return flattened_df

Static decisions:
- 100 trees
- 500 max samples (out of 3900)
- 10 max depth of tree
- 50 max leaf nodes
- 5-fold CV

In [56]:
cv_data1 = grid_search(Xs, Xnames, y,
                       cv_seed=7379, 
                       rf_seed=3551,
                       n_estimators=100,
                       max_features=[10,25,50,100], # what the DIY gridsearch varies on
                       max_samples=500,
                       max_depth=10,
                       max_leaf_nodes=50,
                       cv=5)

Elapsed: 5 m 44 s


In [57]:
cv_data1

,representation,mean_accuracy,mean_sensitivity,mean_specificity,elapsed_seconds,mtry
0,X_bot.npz,0.8944,0.2031,1.0000,4.8,10
1,X_bot_feat.npz,0.9059,0.2902,1.0000,2.2,10
2,X_bot_svd.npz,0.9369,0.5493,0.9962,6.0,10
3,X_bot_svd_cos.npz,0.9354,0.5279,0.9976,5.9,10
4,X_bot_svd_feat.npz,0.9418,0.5822,0.9967,6.0,10
5,X_bot_svd_feat_cos.npz,0.9400,0.5629,0.9976,5.9,10
6,X_bot_tfidf.npz,0.8962,0.2167,1.0000,2.3,10
7,X_bot_tfidf_feat.npz,0.9028,0.2669,1.0000,2.5,10
8,X_bot_tfidf_svd.npz,0.9254,0.4370,1.0000,6.2,10
9,X_bot_tfidf_svd_cos.npz,0.9223,0.4140,1.0000,6.0,10


In [11]:
dir_path = os.path.join("data","3_modeling")
try:
    os.stat(dir_path)
except:
    os.mkdir(dir_path)

In [12]:
#file_path = os.path.join(dir_path, "cv_data1.csv")
#cv_data1.to_csv(file_path, index=False)

---